## Достаем `bboxes` из rs19 для полуавтоматической разметки

In [67]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

import json

import os
from pathlib import Path
import shutil

from pprint import pprint
from tqdm import tqdm

1) игнорировать белый цвет 255 интенс игнорируется example_viz
2) IoU map - maximize


распарсить Uint8 и наложить маски - мультимаска, какие классы за что отвечают в example_viz - матрица - в RLE, ручная аннотация, trackanythingwithDEVA(аналог cvat) - архитектура, pardonizzhopa, roboflow для разметки 11к картинок

чисто код -> трансформеры(без ui) размечают
bboxes -> поиск рельс

убираем из bboxes все что не рельсы

deva video больше картинки и можно d rjyntrcnt kexit gjybvfnm / можно разметить видео, скачать стрелки с разных ракурсов и размечать dfeva/cvat

если не получается полуавтомат - комбинированный - часть  railsem50% - часть из видео50%

Аннотация. хорошо контуры ищутся, когда bboxes чуть шире и есть весь контур

Провести анализ размерности картинок и убрать пиксельные хуйни

In [2]:
with open(r"C:\Users\ivano\OneDrive\Desktop\Синара\Синара\railsem_true\jsons\rs19_val\rs00578.json") as file:
    data = file.read()

dic = json.loads(data)
pprint(dic)

{'frame': 'rs00578',
 'imgHeight': 1080,
 'imgWidth': 1920,
 'objects': [{'boundingbox': [1004, 523, 1028, 532], 'label': 'switch-unknown'},
             {'boundingbox': [930, 518, 946, 524], 'label': 'switch-unknown'},
             {'boundingbox': [957, 535, 987, 549], 'label': 'switch-right'},
             {'boundingbox': [1076, 490, 1084, 503],
              'label': 'track-sign-front'},
             {'boundingbox': [997, 517, 1015, 523], 'label': 'switch-unknown'},
             {'boundingbox': [1054, 547, 1103, 562], 'label': 'switch-left'},
             {'boundingbox': [956, 954, 1333, 1079], 'label': 'switch-left'},
             {'boundingbox': [1256, 553, 1393, 622], 'label': 'buffer-stop'},
             {'label': 'pole',
              'polygon': [[1415, 642],
                          [1415, 516],
                          [1413, 422],
                          [1403, 299],
                          [1391, 202],
                          [1369, 50],
                          [1

### Функция для перебора всех картинок и аннотаций в папках `/images/` and `/annotations/`
### для чтение `bboxes` и вырезания их из оригинальнйо картинки для создания датасета для полуавтомат. разметки полигонов

4 класса переключателей:
 - 'switch-indicator',
 - 'switch-left',
 - 'switch-right',
 - 'switch-static',
 - 'switch-unknown',

In [117]:
path_to_imgs = Path('../../data/jpgs/rs19_val')
path_to_jsons = Path('../../data/jsons/rs19_val')
classname = 'switch'
imgs_list = os.listdir(path_to_imgs)    
jsons_list = os.listdir(path_to_jsons)

for filename in tqdm(jsons_list, 'Обработано файлов'):
    with open(Path(path_to_jsons, filename)) as file:
        data = json.loads(file.read())

    img_path = str(Path(path_to_imgs, filename[:-4] + 'jpg'))
    img = cv2.imread(img_path)

    for idx, obj in enumerate(data['objects']):
        if classname in obj['label']:
            try:
                x, y, w, h = obj['boundingbox']
                
                roi_color = img[y:h, x:w]   #взять больше bboxes
                cv2.imwrite(f'../../data/bboxes_switches/{data["frame"]}_{idx}.jpg', roi_color)
            except: 
                continue

Обработано файлов:   0%|          | 0/8500 [00:00<?, ?it/s]

Обработано файлов: 100%|██████████| 8500/8500 [04:07<00:00, 34.28it/s]


In [ ]:
path_to_imgs = Path('../../data/jpgs/rs19_val')
path_to_jsons = Path('../../data/jsons/rs19_val')
classname = 'switch'
# imgs_list = os.listdir(path_to_imgs)   #disabled
jsons_list = os.listdir(path_to_jsons)
target_dir = Path('../../data/bboxes_switches/')


def cropp_bboxes_from_classname_railsem(path_imgs: Path, path_jsons: Path, 
                                        classname: str, jsons_list: list, target_dir: Path):
    """
    Вырезаем bboxes из картинок датасета railsem19 и сохраняем в target_dir
    bboxes from jsons/
    imgs from jpgs/

    output: list of rsxxxxx_idx (frame_object)
    """
    for filename in tqdm(jsons_list, 'Обработано файлов'):
        with open(Path(path_jsons, filename)) as file:
            data = json.loads(file.read())

        img_path = str(Path(path_imgs, filename[:-4] + 'jpg'))
        img = cv2.imread(img_path)

        for idx, obj in enumerate(data['objects']):
            if classname in obj['label']:
                try:
                    x, y, w, h = obj['boundingbox']
                    roi_color = img[y:y+h, x:x+w]
                    cv2.imwrite(Path(target_dir, f'{data["frame"]}_{idx}.jpg'), roi_color)
                except: 
                    continue

In [113]:
cropp_bboxes_from_classname_railsem(
    path_to_imgs,
    path_to_jsons,
    classname,
    jsons_list,
    target_dir
)

Обработано файлов:   3%|▎         | 274/8500 [00:07<03:52, 35.40it/s]


KeyboardInterrupt: 

In [111]:
len(os.listdir(r'C:\Users\ivano\OneDrive\Документы\GitHub\Rail_segm\data\bboxes_switches'))

0

11053

In [102]:
shutil.rmtree(target_dir)
os.makedirs(target_dir)

In [ ]:
# cv2.imshow('name', roi_color)
# cv2.waitKey(0)
# cv2.destroyAllWindows()